In [1]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig

In [2]:
dataset = load_dataset("iamjoon/finance_news_summarizer", split="train")

README.md:   0%|          | 0.00/781 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/991 [00:00<?, ? examples/s]

In [3]:
print("전체 데이터 크기:", len(dataset))

전체 데이터 크기: 991


In [4]:
test_ratio = 0.5

train_data = []
test_data = []

In [5]:
# 데이터셋 인덱스를 0부터 데이터 개수-1까지 리스트로 만든다
data_indices = list(range(len(dataset)))
# 테스트 사이즈를 구한다
test_size = int(len(dataset) * test_ratio)

# 데이터셋 인덱스 지정
test_data = data_indices[:test_size]
train_data = data_indices[test_size:]

In [6]:
# openai format으로 데이터 변환
def format_data(sample):
    return {
        "messages":[
            {
                "role": "system",
                "content": sample["system_prompt"]
            },
            {
                "role": "user",
                "content": sample["user_prompt"]
            },
            # 정답 예시 제공 json타입이기 때문에 str타입으로 변환 후 제공
            {
                "role": "assistant",
                "content": str(sample["assistant"])
            }
        ]
    }

In [7]:
# 데이터셋 OpenAI format으로 변환
train_dataset = [format_data(dataset[i]) for i in train_data]
test_dataset = [format_data(dataset[i]) for i in test_data]

In [8]:
# 최종 데이터셋 크기 출력
print("훈련 데이터 크기:", len(train_dataset))
print("테스트 데이터 크기:", len(test_dataset))

훈련 데이터 크기: 496
테스트 데이터 크기: 495


In [9]:
train_dataset[345]["messages"]

[{'role': 'system',
  'content': '당신은 주어진 뉴스로부터 종목에 영향을 주는 뉴스인지 판별하는 금융 뉴스 판별기입니다.\n두 가지 답변 케이스가 존재하며 무조건 파이썬의 dictionary 형식으로 작성하십시오.\n큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다. 따라서 주의하십시오.\n아래 dictionary에서 각 value는 지시사항에 해당합니다. 지사사항을 따라 적지마십시오. 해당 지시사항에 따라 적절한 value를 채워넣으십시오.\n해당사항이 없다면 빈 문자열 또는 빈 리스트로 적어야 합니다. 임의로 \'없음\' 등을 적어서는 안 됩니다.\n\n만약 해당 뉴스가 특정 종목(회사)이 언급되지 않거나, 특정 종목(회사)와 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.\n\n답변:\n{"is_stock_related": False,\n"summary": "여기에는 해당 뉴스를 요약해서 요약문을 작성하십시오"}\n\n만약 해당 뉴스가 특정 종목(회사)들과 연관되었거나, 특정 종목(회사)과 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.\n\n답변:\n{"is_stock_related": True,\n"positive_impact_stocks": ["파이썬 문자열 리스트의 형태로 이 뉴스가 긍정적인 영향을 줄것으로 추정되는 종목들의 이름을 작성하십시오. 약자로 적거나 별명으로 적지마십시오. 종목명으로 추정되는 한글명을 적으십시오. 뉴스로부터 추정할 수 있는 정확한 풀네임으로 적으십시오. 만약, 존재하지 않는다면 빈 리스트로 작성하십시오."],\n"reason_for_positive_impact": "위의 종목들이 해당 뉴스로부터 긍정적인 영향을 받을 것으로 추정한 이유를 여기에다가 작성하십시오",\n"positive_keywords": ["긍정적인 영향을 줄 것으로 추정되는 종목들이 존재했다면 여기에 긍정적인 영향을 주는데 근거가 

In [10]:
# 리스트 형태에서 다시 Dataset 객체로 변경
print(type(train_dataset))
print(type(test_dataset))
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)
print(type(train_dataset))
print(type(test_dataset))

<class 'list'>
<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [11]:
# 허깅페이스 모델 ID
model_id = "NCSoft/Llama-VARCO-8B-Instruct"

# device_map="auto": 자동으로 적절한 디바이스(GPU/CPU)에 배치
# torch_dtype=torch.bfloat16: 메모리 효율성을 위해 bfloat16 타입 사용
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    # load_in_8bit=True, 
    torch_dtype=torch.bfloat16,
)
# model id 바탕으로 사전학습 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

In [12]:
# 템플릿 적용
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 뉴스로부터 종목에 영향을 주는 뉴스인지 판별하는 금융 뉴스 판별기입니다.
두 가지 답변 케이스가 존재하며 무조건 파이썬의 dictionary 형식으로 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다. 따라서 주의하십시오.
아래 dictionary에서 각 value는 지시사항에 해당합니다. 지사사항을 따라 적지마십시오. 해당 지시사항에 따라 적절한 value를 채워넣으십시오.
해당사항이 없다면 빈 문자열 또는 빈 리스트로 적어야 합니다. 임의로 '없음' 등을 적어서는 안 됩니다.

만약 해당 뉴스가 특정 종목(회사)이 언급되지 않거나, 특정 종목(회사)와 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.

답변:
{"is_stock_related": False,
"summary": "여기에는 해당 뉴스를 요약해서 요약문을 작성하십시오"}

만약 해당 뉴스가 특정 종목(회사)들과 연관되었거나, 특정 종목(회사)과 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.

답변:
{"is_stock_related": True,
"positive_impact_stocks": ["파이썬 문자열 리스트의 형태로 이 뉴스가 긍정적인 영향을 줄것으로 추정되는 종목들의 이름을 작성하십시오. 약자로 적거나 별명으로 적지마십시오. 종목명으로 추정되는 한글명을 적으십시오. 뉴스로부터 추정할 수 있는 정확한 풀네임으로 적으십시오. 만약, 존재하지 않는다면 빈 리스트로 작성하십시오."],
"reason_for_positive_impact": "위의 종목들이 해당 뉴스로부터 긍정적인 영향을 받을 것으로 추정한 이유를 여기에다가 작성하십시오",
"positive_keywords": ["긍정적인 영향을 줄 것으로 추정되는 종목들이 존재했다면 여기에 긍정적인 영향을 주

In [13]:
# LoRA 설정
peft_config = LoraConfig(
    lora_alpha=32,           # LoRA 스케일링 파라미터
    lora_dropout=0.1,        # 드롭아웃 비율
    r=8,                     # rank (저차원 분해의 차원)
    bias="none",             # 바이어스 학습 여부
    target_modules=["q_proj", "v_proj"],  # LoRA를 적용할 모듈들
    task_type="CAUSAL_LM",   # 태스크 타입 (언어모델링)
)

In [14]:
from trl import SFTConfig

args = SFTConfig(
    output_dir="llama3-8b-summarizer-ko",          # 저장될 디렉토리와 저장소 ID
    num_train_epochs=3,                            # 훈련 에포크 수 
    per_device_train_batch_size=2,                 # GPU당 배치 크기
    gradient_accumulation_steps=2,                 # 그래디언트 누적 스텝
    gradient_checkpointing=True,                   # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",                     # 최적화기
    logging_steps=10,                              # 로깅 주기
    save_strategy="steps",                         # 저장 전략
    save_steps=50,                                 # 저장 주기
    bf16=True,                                     # bfloat16 사용
    learning_rate=1e-4,                            # 학습률
    max_grad_norm=0.3,                             # 그래디언트 클리핑
    warmup_ratio=0.03,                             # 워밍업 비율
    lr_scheduler_type="constant",                  # 스케줄러 유형
    push_to_hub=False,                             # 허브 업로드 안 함
    remove_unused_columns=False,                   # 사용하지 않는 컬럼 제거 안 함
    dataset_kwargs={"skip_prepare_dataset": True}, # 데이터셋 전처리 스킵
    report_to=None                                 # 로깅 툴 사용 안 함
)

In [15]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for example in batch:
        messages = example["messages"]

        prompt = "<|begin_of_text|>"
        for msg in messages:
            role = msg["role"]
            content = msg["content"].strip()
            prompt += f"<|start_header_id|>{role}<|end_header_id|>\n{content}<|eot_id|>"

        text = prompt.strip()

        # 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        labels = [-100] * len(input_ids)

        # assistant 응답의 시작 위치 찾기
        assistant_header = "<|start_header_id|>assistant<|end_header_id|>\n"
        assistant_tokens = tokenizer.encode(assistant_header, add_special_tokens=False)
        eot_token = "<|eot_id|>"
        eot_tokens = tokenizer.encode(eot_token, add_special_tokens=False)

        # 레이블 범위 지정
        i = 0
        while i <= len(input_ids) - len(assistant_tokens):
            if input_ids[i:i + len(assistant_tokens)] == assistant_tokens:
                start = i + len(assistant_tokens)
                end = start
                while end <= len(input_ids) - len(eot_tokens):
                    if input_ids[end:end + len(eot_tokens)] == eot_tokens:
                        break
                    end += 1
                for j in range(start, end):
                    labels[j] = input_ids[j]
                for j in range(end, end + len(eot_tokens)):
                    labels[j] = input_ids[j]  # <|eot_id|> 토큰도 포함
                break
            i += 1

        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    max_length = max(len(ids) for ids in new_batch["input_ids"])
    for i in range(len(new_batch["input_ids"])):
        pad_len = max_length - len(new_batch["input_ids"][i])
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * pad_len)
        new_batch["attention_mask"][i].extend([0] * pad_len)
        new_batch["labels"][i].extend([-100] * pad_len)

    for k in new_batch:
        new_batch[k] = torch.tensor(new_batch[k])

    return new_batch

In [16]:
max_seq_length=8192

example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 2884])
어텐션 마스크 형태: torch.Size([1, 2884])
레이블 형태: torch.Size([1, 2884])


In [17]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

입력에 대한 정수 인코딩 결과:
[128000, 128006, 9125, 128007, 198, 65895, 83628, 34804, 56773, 125441, 111068, 25941, 123103, 99458, 88708, 19954, 126652, 56773, 16969, 111068, 25941, 117469, 106478, 102517, 44005, 104193, 123061, 111068, 25941, 106478, 102517, 21121, 80052, 627, 103097, 109521, 111964, 127491, 122625, 119097, 108859, 101480, 93917, 101868, 56069, 13094, 168, 235, 105, 21028, 11240, 106612, 77437, 43139, 114839, 16582, 119978, 627, 65993, 108, 103386, 36092, 112, 102260, 109055, 19954, 105642, 103386, 36092, 112, 102260, 105880, 103607, 34609, 113348, 45618, 49085, 88525, 96677, 119978, 13, 127063, 11240, 56069, 113890, 18359, 62085, 99742, 102893, 105365, 102467, 112215, 124005, 13, 106725, 27796, 56773, 21028, 16582, 119978, 627, 54059, 54542, 11240, 57575, 106603, 907, 16969, 67890, 30426, 115790, 19954, 95713, 61938, 13, 67890, 56154, 115790, 18359, 106725, 103607, 22035, 100711, 119978, 13, 95713, 67890, 30426, 115790, 19954, 106725, 103607, 104834, 24486, 907, 18918, 104965, 

In [18]:
decoded_text = tokenizer.decode(
    batch["input_ids"][0].tolist(),
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\ninput_ids 디코딩 결과:")
print(decoded_text)


input_ids 디코딩 결과:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
당신은 주어진 뉴스로부터 종목에 영향을 주는 뉴스인지 판별하는 금융 뉴스 판별기입니다.
두 가지 답변 케이스가 존재하며 무조건 파이썬의 dictionary 형식으로 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다. 따라서 주의하십시오.
아래 dictionary에서 각 value는 지시사항에 해당합니다. 지사사항을 따라 적지마십시오. 해당 지시사항에 따라 적절한 value를 채워넣으십시오.
해당사항이 없다면 빈 문자열 또는 빈 리스트로 적어야 합니다. 임의로 '없음' 등을 적어서는 안 됩니다.

만약 해당 뉴스가 특정 종목(회사)이 언급되지 않거나, 특정 종목(회사)와 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.

답변:
{"is_stock_related": False,
"summary": "여기에는 해당 뉴스를 요약해서 요약문을 작성하십시오"}

만약 해당 뉴스가 특정 종목(회사)들과 연관되었거나, 특정 종목(회사)과 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.

답변:
{"is_stock_related": True,
"positive_impact_stocks": ["파이썬 문자열 리스트의 형태로 이 뉴스가 긍정적인 영향을 줄것으로 추정되는 종목들의 이름을 작성하십시오. 약자로 적거나 별명으로 적지마십시오. 종목명으로 추정되는 한글명을 적으십시오. 뉴스로부터 추정할 수 있는 정확한 풀네임으로 적으십시오. 만약, 존재하지 않는다면 빈 리스트로 작성하십시오."],
"reason_for_positive_impact": "위의 종목들이 해당 뉴스로부터 긍정적인 영향을 받을 것으로 추정한 이유를 여기에다가 작성하십시오",
"positive_keywords": ["긍정적인 영향을 줄 것으로 추정되는 종목들이 존재

In [19]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

레이블에 대한 정수 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

In [20]:
trainer = SFTTrainer(
    model=model,
    args=args,
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이 설정
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

/root/.local/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/root/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [21]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

/root/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


/root/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/root/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/root/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/root/.local/lib/python3.10/

In [22]:
prompt_lst = []
label_lst = []

for messages in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    input = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[0] + '<|start_header_id|>assistant<|end_header_id|>\n'
    label = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[1].split('<|eot_id|>')[0]
    prompt_lst.append(input)
    label_lst.append(label)

In [23]:
print(prompt_lst[200])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 뉴스로부터 종목에 영향을 주는 뉴스인지 판별하는 금융 뉴스 판별기입니다.
두 가지 답변 케이스가 존재하며 무조건 파이썬의 dictionary 형식으로 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다. 따라서 주의하십시오.
아래 dictionary에서 각 value는 지시사항에 해당합니다. 지사사항을 따라 적지마십시오. 해당 지시사항에 따라 적절한 value를 채워넣으십시오.
해당사항이 없다면 빈 문자열 또는 빈 리스트로 적어야 합니다. 임의로 '없음' 등을 적어서는 안 됩니다.

만약 해당 뉴스가 특정 종목(회사)이 언급되지 않거나, 특정 종목(회사)와 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.

답변:
{"is_stock_related": False,
"summary": "여기에는 해당 뉴스를 요약해서 요약문을 작성하십시오"}

만약 해당 뉴스가 특정 종목(회사)들과 연관되었거나, 특정 종목(회사)과 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.

답변:
{"is_stock_related": True,
"positive_impact_stocks": ["파이썬 문자열 리스트의 형태로 이 뉴스가 긍정적인 영향을 줄것으로 추정되는 종목들의 이름을 작성하십시오. 약자로 적거나 별명으로 적지마십시오. 종목명으로 추정되는 한글명을 적으십시오. 뉴스로부터 추정할 수 있는 정확한 풀네임으로 적으십시오. 만약, 존재하지 않는다면 빈 리스트로 작성하십시오."],
"reason_for_positive_impact": "위의 종목들이 해당 뉴스로부터 긍정적인 영향을 받을 것으로 추정한 이유를 여기에다가 작성하십시오",
"positive_keywords": ["긍정적인 영향을 줄 것으로 추정되는 종목들이 존재했다면 여기에 긍정적인 영향을 주

In [24]:
print(label_lst[200])


{'is_stock_related': True, 'negative_impact_stocks': [], 'negative_keywords': [], 'positive_impact_stocks': ['국민은행', '우리은행', '부산은행', '수협은행', '하나은행'], 'positive_keywords': ['HUG', '표준 프로젝트 파이낸싱', '주택사업금융', '후분양'], 'reason_for_negative_impact': '', 'reason_for_positive_impact': '이 뉴스는 HUG가 주택사업자에게 자금조달을 지원하는 표준 PF 및 후분양 표준 PF 보증의 주관금융사로 선정되었다는 내용으로, 이는 관련 은행들에게 주택건설 관련 금융 지원 기회를 제공함으로써 긍정적인 영향을 미칠 수 있다.', 'summary': 'HUG가 주택사업자 자금조달 지원을 위한 표준 프로젝트 파이낸싱의 주관금융사로 국민은행, 우리은행, 부산은행, 수협은행, 하나은행을 선정하여 저금리 금융지원을 통해 주택 사업자의 비용 증가 부담 완화에 기여할 예정이다.'}


In [25]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import  AutoTokenizer, pipeline

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [26]:
peft_model_id = "llama3-8b-summarizer-ko/checkpoint-372"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

In [27]:
eos_token = tokenizer("<|eot_id|>",add_special_tokens=False)["input_ids"][0]

In [28]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [29]:
for prompt, label in zip(prompt_lst[10:15], label_lst[10:15]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


    response:
{'is_stock_related': True, 'negative_impact_stocks': ['애플', '엔비디아', 'TSMC'], 'negative_keywords': ['스마트폰 판매 감소', '반도체 업체', '애플', '엔비디아', 'TSMC'], 'positive_impact_stocks': [], 'positive_keywords': [],'reason_for_negative_impact': '가트너가 전세계 스마트폰 판매량이 7% 감소할 것으로 전망하면서 애플을 비롯한 스마트폰 제조사와 반도체 업체인 엔비디아와 TSMC에 부정적인 영향을 미칠 것으로 예상됩니다.','reason_for_positive_impact': '','summary': '가트너는 전세계 스마트폰 판매량이 올해 7% 감소할 것으로 전망하며, 이는 애플과 같은 스마트폰 제조사 및 엔비디아와 TSMC와 같은 반도체 업체에 부정적인 영향을 미칠 것으로 예상됩니다. 또한, 유럽연합은 가상자산을 이용한 돈세탁을 막기 위해 관련 기업을 규제하는 방안에 잠정 합의하였습니다. 스피릿 항공은 프론티어 항공과의 합병을 연기하고 제트블루 항공과의 합병 가능성을 검토 중이며, 텐센트와 바이트댄스는 하반기 대규모 구조조정을 준비하고 있습니다.'}
    label:

{'is_stock_related': True, 'negative_impact_stocks': ['애플', '엔비디아', 'TSMC', '텐센트', '바이트댄스'], 'negative_keywords': ['스마트폰 판매 감소', '인력 감축', '비용 절감', '경제 둔화'], 'positive_impact_stocks': [], 'positive_keywords': [], 'reason_for_negative_impact': '가트너의 보고서에 따르면 스마트폰 판매량의 감소는 애플과 같은 제조업체뿐만 아니라 엔비디아와 TSMC 같은 반도체 업체에게 부정적인 영향을 미칠 것으로 보입니다. 또한, 텐센트와 